## Module for building Henderson's Mixed Model Equations for single trait including marker  and polygenic effect



### <font color="red"> Data</font>

In [1]:
using DataFrames
using Distributions

In [2]:
FILE = "../../data/small.ped";

In [3]:
Animal = ["S1","D1","O1","O3"]
y = [100.0, 50.0, 150.0, 40.0];
df = DataFrame(Animal=Animal,y=y);

In [4]:
srand(123)
d = Binomial(2,0.5)
nObs     = 4
nMarkers = 5
M        = float(rand(d,(nObs,nMarkers)));

In [5]:
df=[df DataFrame(M)]

,Animal,y,x1,x2,x3,x4,x5
1,S1,100.0,1.0,0.0,1.0,1.0,1.0
2,D1,50.0,2.0,0.0,2.0,2.0,1.0
3,O1,150.0,1.0,2.0,0.0,1.0,0.0
4,O3,40.0,0.0,0.0,2.0,1.0,1.0


### <font color="red"> Run module</font>

In [6]:
MODULE_PATH="../../Module/MME.jl";
include(MODULE_PATH);

In [7]:
varRes = 5.0
varGen = 5.0
markerProp = 0.8
MVarGen= varGen*markerProp
PVarGen= varGen*(1-markerProp);

In [8]:
ped = PedModule.mkPed(FILE);

In [9]:
varRes = 5.0
mme = MMEModule.initMME("y = intercept + Animal",varRes);

In [10]:
G = reshape([PVarGen],1,1)
MMEModule.setAsRandom(mme,"Animal",ped,G)

### <font color="blue"> Gauss-Seidel</font> 

In [11]:
resG = MMEModule.getSolG(mme,df,outFreq=100)

6x2 Array{Any,2}:
 "intercept: intercept"  84.5242     
 "Animal: S1"             4.16646    
 "Animal: D1"            -4.16707    
 "Animal: O1"             5.95206    
 "Animal: O3"            -4.04794    
 "Animal: O2"            -0.000304593

### <font color="blue"> Gibbs</font> 

In [12]:
sol  = fill(0.0,size(mme.mmeLhs,1));
res2 = MMEModule.getSolGibbs(mme,df,outFreq=10000)

at sample: 10000
at sample: 20000
at sample: 30000
at sample: 40000

6x2 Array{Any,2}:
 "intercept: intercept"  84.5217   
 "Animal: S1"             4.17441  
 "Animal: D1"            -4.16322  
 "Animal: O1"             5.95959  
 "Animal: O3"            -4.04315  
 "Animal: O2"             0.0073063


at sample: 50000


### <font color="red"> work with markers</font>

### add marker information

In [13]:
function addMarkers(mme,df,G::Float64)
    M = convert(Array,df)
    mme.M = MarkerMatrix(M,G)
end

addMarkers (generic function with 1 method)

##### functions for markers

In [14]:
function get_column(X,j)
    nrow,ncol = size(X)
    if j>ncol||j<0
        error("column number is wrong!")
    end
    indx = 1 + (j-1)*nrow
    ptr = pointer(X,indx)
    pointer_to_array(ptr,nrow)
end

function get_column_ref(X)
    ncol = size(X)[2]
    xArray = Array(Array{Float64,1},ncol)
    for i=1:ncol
        xArray[i] = get_column(X,i)
    end
    return xArray
end

function center!(X)
    nrow,ncol = size(X)
    colMeans = mean(X,1)
    BLAS.axpy!(-1,ones(nrow)*colMeans,X)
    return colMeans
end

center! (generic function with 1 method)

##### types for markers

In [15]:
type MarkerMatrix
    X::Array{Float64,2}
    xArray::Array{Array{Float64,1},1}
    markerMeans::Array{Float64,2}
    mean2pq::Float64
    G::Float64
    centered::Bool
    function MarkerMatrix(X::Array{Float64,2},G::Float64)
        markerMeans = center!(X) #centering
        p           = markerMeans/2.0
        mean2pq     = (2*p*(1-p)')[1,1]
        xArray      = get_column_ref(X)
        new(X,xArray,markerMeans,mean2pq,G,true)
    end
end

### Gibbs sampler

In [16]:
function sampleMCMC(nIter,mme,df;outFreq=100)
    if size(mme.mmeRhs)==() 
        MMEModule.getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    sol = zeros(p)
    solMean = zeros(p)
    
    vEff = mme.M.G/mme.M.mean2pq 
    vRes = mme.R


    numMarker = size(mme.M.X,2)
    α  = zeros(Float64,numMarker)
    meanAlpha = zeros(Float64,numMarker)
    mArray = mme.M.xArray
    mpm = [dot(mme.M.X[:,i],mme.M.X[:,i]) for i=1:size(mme.M.X,2)]   
    ycorr = vec(full(mme.ySparse))
    M = mme.M.X
    
    for iter=1:nIter
        #sample non-marker part
        ycorr = ycorr + mme.X*sol
        rhs = mme.X'ycorr #

        MMEModule.Gibbs(mme.mmeLhs,sol,rhs,vRes)
        ycorr = ycorr - mme.X*sol

        solMean += (sol - solMean)/iter
        
        #sample marker
        MMEModule.sample_effects_ycorr!(M,mArray,mpm,ycorr,α,meanAlpha,vRes,vEff,iter)

        if iter%outFreq==0
            println("at sample: ",iter)
        end
    end
    output = Dict()
    output["posteriorMeanLocationParms"] = [MMEModule.getNames(mme) solMean]
    output["posteriorMeanMarkerEffects"] = meanAlpha
    return output
end

sampleMCMC (generic function with 1 method)

### run it

In [17]:
addMarkers(mme,df[:,3:7],MVarGen);

In [28]:
output=sampleMCMC(50000,mme,df,outFreq=10000)

at sample: 10000
at sample: 20000
at sample: 30000
at sample: 40000
at sample: 50000

Dict{Any,Any} with 2 entries:
  "posteriorMeanLocationP… => 6x2 Array{Any,2}:…
  "posteriorMeanMarkerEff… => [3.025238422521124,12.260743992728305,-17.3593660…

In [29]:
output["posteriorMeanLocationParms"]

6x2 Array{Any,2}:
 "intercept: intercept"  85.0754    
 "Animal: S1"             2.63181   
 "Animal: D1"            -2.64211   
 "Animal: O1"             1.74085   
 "Animal: O3"            -2.04497   
 "Animal: O2"            -0.00768987

In [30]:
output["posteriorMeanMarkerEffects"]

5-element Array{Float64,1}:
   3.02524
  12.2607 
 -17.3594 
  -4.10988
  -6.13187